In [1]:
#%pip install voila
#%pip install papermill

In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from get_stackexchange_sites import stackexchange_sites
from notebooks import *

# Widgets
dropdown_sites = widgets.Dropdown(
    options=stackexchange_sites,
    value=stackexchange_sites[0],
    description='StackExchange Site:',
    disabled=False,
    style={'description_width': 'initial'}
)

question_input = widgets.Textarea(
    value='',
    placeholder='Posez votre question ici....',
    description='Question:',
    disabled=False,
    layout=widgets.Layout(width='90%', height='80px')
)

recommended_answers = widgets.Output()

train_button = widgets.Button(
    description='Entraîner le modèle',
    button_style='success',
    layout=widgets.Layout(width='90%')
)

results_output = widgets.Output()

In [3]:

# Functions to execute notebooks
def run_notebook(path, params):
    with open(path, encoding='utf-8') as f: 
        nb = nbformat.read(f, as_version=4)
        ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
        nb.cells.insert(0, nbformat.v4.new_code_cell(source=params))
        ep.preprocess(nb, {'metadata': {'path': '.'}})
    return nb

# Sequential execution handlers
def collect_data(site):
    params = f"site = '{dropdown_sites.value}'"
    run_notebook('APIdataExtraction.ipynb', params)
    preprocess_data()

def preprocess_data():
    run_notebook('DataPreprocessing.ipynb', '')
    cluster_data()

def cluster_data():
    run_notebook('Clustering.ipynb', '')

def train_model(change):
    run_notebook('RecommendationModel.ipynb', '')
    with results_output:
        clear_output()
        display(HTML("<h3>Modèle entraîné avec succès!</h3>"))

def recommend_answers(change):
    user_question = question_input.value
    params = f"user_question = '''{user_question}'''"
    run_notebook('recommendation.ipynb', params)
    with recommended_answers:
        clear_output()
        display(HTML("<h3>Réponses recommandées :</h3>"))
        # Assume the last cell of recommendation.ipynb writes the recommended answers to a variable `recommended`
        # You can access it here and display it
        display(recommended)


In [4]:

# Bind functions to events
dropdown_sites.observe(collect_data, names='value')
train_button.on_click(train_model)
question_input.observe(recommend_answers, names='value')

# Layout
site_section = widgets.VBox([widgets.HTML("<h2>Sélectionnez le site</h2>"), dropdown_sites], layout=widgets.Layout(padding='20px'))
data_section = widgets.VBox([widgets.HTML("<h2>Données</h2>"), recommended_answers], layout=widgets.Layout(padding='20px'))
question_section = widgets.VBox([widgets.HTML("<h2>Posez votre question</h2>"), question_input], layout=widgets.Layout(padding='20px'))
train_section = widgets.VBox([widgets.HTML("<h2>Entraînement du modèle</h2>"), train_button, results_output], layout=widgets.Layout(padding='20px'))


In [7]:
left_panel = widgets.VBox([site_section, data_section])
center_panel = question_section
right_panel = train_section

dashboard = widgets.HBox([left_panel, center_panel, right_panel], layout=widgets.Layout(width='100%', display='flex', justify_content='space-around'))

display(HTML("<style> .widget-hbox { justify-content: space-around; } .widget-vbox { align-items: center; } .widget-button { background-color: #28a745; color: white; } </style>"))
display(dashboard)

collect_data(site=dropdown_sites.value)